Anticipez les besoins en consommation électrique de bâtiments
=============================================================

![logo-seattle](../reports/figures/logo-seattle.png)


Explication des variables:
[City of seattle](https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

from src.utils.univar import UnivariateAnalysis
from src.utils.bivar import BivariateAnalysis


In [ ]:
data = pd.read_pickle('../data/interim/full_data.pickle')

In [ ]:
data.columns

Variable à prédire (targets)

   * SiteEnergyUse/WN (wether normalized?)
   * TotalGHGEmissions

Variables sélectionnées comme entrée du modèle
    
   * Surface du bâtiment
   * Premiere utilisation du bâtiment
   * Surface destinee au premier usage du batiment
    

## Quelles sont les variables corrélées avec la cible?

In [ ]:
bivar = BivariateAnalysis(data)

In [ ]:
bivar.anova('SiteEUIWN_kBtu_sf', 'LargestPropertyUseTypeNorm')

In [ ]:
bivar.anova('SiteEnergyUse_kBtu', 'LargestPropertyUseTypeNorm')

In [ ]:
bivar.regression(['PropertyGFATotal', 'SiteEnergyUseWN_kBtu'])

In [ ]:
sns.scatterplot(x='PropertyGFATotal', 
                y='SiteEnergyUse_kBtu',
                data=data, hue='LargestPropertyUseTypeNorm',
                palette=['#f5aa42', 
                         '#69f542',
                         '#f54242'])

In [ ]:
use_type = "LargestPropertyUseTypeNorm"
norm_ = "normal_demand_use_type"

In [ ]:
bivar = BivariateAnalysis(data[data[use_type] == norm_])

In [ ]:
bivar.regression(['PropertyGFATotal', 'SiteEnergyUseWN_kBtu'])